In [1]:
import warnings
import subprocess
import os
from selenium import webdriver
import time
from fast_reports import *

start = time.time()

warnings.filterwarnings('ignore')
checkfilepath = r"C:\Users\016434613\Downloads\progress_dt_tasks_details_report.xlsx"
exists = False

''' This sentence takes care of deleting old tasks report on downloads folder '''
subprocess.run("del /f C:\\Users\\016434613\\Downloads\\progress_dt_tasks_details_report.xlsx", shell=True, capture_output=True)

#options = webdriver.ChromeOptions()
#options.add_argument("headless")

''' Creating the driver conection to handle automatic session with Selenium '''
mydriver = webdriver.Chrome("chromedriver.exe")

''' automatically opening the above created session with the link in between parenthesis '''
mydriver.get("https://progress.us1a.cirrus.ibm.com/api/rpt/dt/1600277460000/ms/jas8n38daai/1/1600431650693")

''' check if the path of file exists in the directory to make sure file was actually downloaded '''
while not exists:
    time.sleep(1)
    exists = os.path.exists(checkfilepath)

''' Close the session driver when the file was downloaded '''
mydriver.close()

backlogsub()
specsub()
changesub()

download_master()
delivery_report()
validsub()

dev_over90()
chg_over30()

download_ideas_sd()
ideas()

update_specification()

end = time.time()
mins = (end-start)/60.00
if mins < 1:
    mins = end-start
    minsstr = "Reports created/updated in: {:.2f} seconds".format(mins) 
    print(minsstr)
else:
    minsstr = "Reports created/updated in: {:.2f} minutes".format(mins)
    print(minsstr)

Reports created/updated in: 13.19 minutes


In [ ]:
from fast_reports import *
import warnings
warnings.filterwarnings('ignore')

dev_over90()

## Backlog Only:

In [5]:
import pandas as pd
import numpy as np
import glob
import re
from urllib import request as req
import json

import warnings
warnings.filterwarnings('ignore')

filepath_= r"C:\Users\016434613\Downloads\progress_dt_tasks_details_report.xlsx"

df_Back = pd.read_excel(filepath_,sheet_name="Tasks")
df_Back = df_Back.rename(columns={'DSL/SUPERVISOR (NAME)':'DSL','ROBOT NUMBER':'ROBOT_NUMBER','AUTOMATION TYPE':'AUTOMATION_TYPE','MANUAL EXECUTION DURATION (hours per year)':'SAVING HS'})
df_Back = df_Back.loc[:,['TASK ID','CERTIFIED','DSL','STATUS','ROBOT_NUMBER','TITLE','AUTOMATION_TYPE','REGIONS','SAVING HS']]
df_Back = df_Back.loc[df_Back['STATUS']=="Ready for development"]
df_Back['CERTIFIED'] = df_Back['CERTIFIED'].apply(lambda x: str(x))
df_Back['CERTIFIED'] = df_Back['CERTIFIED'].apply(lambda x: x[4:7]+" "+x[11:15])
df_Back = df_Back.loc[df_Back['ROBOT_NUMBER'].isnull()==True]
df_Back.drop(columns=['ROBOT_NUMBER','STATUS'],inplace=True)
df_Back['Count']=1
df_Back.to_excel(r"C:\Users\016434613\Desktop\Backlog.xlsx", index=False)
print("Report successfully created!")

Report successfully created!


### Older Version of Delivery Report:

In [ ]:
    df = pd.read_excel(filepath_,sheet_name="DT Tasks")
df = df.rename(columns={'ROBOT NUMBER':'ROBOT_NUMBER'})

df_lindo = df.loc[:,['ORIGIN RELEASE','STATUS','ROBOT_NUMBER','TITLE','AUTOMATION TYPE','REGIONS','MANUAL EXECUTION DURATION (hours per year)']]
df_lindo = df_lindo.rename(columns={'MANUAL EXECUTION DURATION (hours per year)':'SAVING HS'})

df_ATA = pd.read_excel(r"C:\Users\016434613\Box\ATA Other\ATA-ODA-WBA-RPA number assigment.xlsx",sheet_name="ATA")
df_ATA = df_ATA.rename(columns={'Task #':'ROBOT_NUMBER'})


df3 = df_lindo
df_ATA_2 = df_ATA.loc[:,['ROBOT_NUMBER','Developer']]

df4 = df3.merge(df_ATA_2, on="ROBOT_NUMBER", how="left")
df4['ORIGIN RELEASE'] = df4['ORIGIN RELEASE'].apply(lambda x: str(x))
df4['QUARTER'] = df4['ORIGIN RELEASE'].apply(lambda x: x[4:7]+x[11:15])

df4['QUARTER'] = df4['QUARTER'].apply(lambda x: '1Q' if x == 'Jan2022' else '1Q' if x == 'Feb2022' else '1Q' if x == 'Mar2022' else 'Other')

df5 = df4.loc[df4['QUARTER']=='1Q',]
df5.drop(columns=["ORIGIN RELEASE"],inplace=True)


df_ODA = pd.read_excel(r"C:\Users\016434613\Box\ATA Other\ATA-ODA-WBA-RPA number assigment.xlsx",sheet_name="ODA")
df_ODA = df_ODA.rename(columns={'Task #':'ROBOT_NUMBER'})
df_ODA = df_ODA.loc[:,['ROBOT_NUMBER','Developer']]
df5 = df5.merge(df_ODA, on="ROBOT_NUMBER", how="left")

df5['Developer'] = df5.apply(lambda x: x['Developer_x'] if str(x['Developer_y'])=="nan" else x['Developer_y'],axis=1)
df5.drop(columns=['Developer_y','Developer_x'],inplace=True)



mgrserials = ['016434613','A01130693','A51733693','900688897']
mgrname = {'016434613':'Martin Campos','A01130693':'Marek Tarkos','A51733693':'Andrej Csiaki','900688897':'Melissa Bledsoe'}
devnames = {}
for mgr in mgrserials:
    request = req.urlopen("https://unified-profile-api.us-south-k8s.intranet.ibm.com/v3/profiles/" + mgr + "/teamResolved")
    datos = request.read().decode()
    dictdatos = json.loads(datos)
    devlist = dictdatos['content']['functional']['reports']
    for dev in devlist:
        #method setdefault(key,[]) sirve para poner una primera key con una lista vacia, si no, tira error de que no existe la key
        devnames.setdefault(mgrname[mgr], []).append(dev['nameDisplay'])
        
### Create Lists ###
devlistmc = devnames['Martin Campos']
devlistmt = devnames['Marek Tarkos']
devlistac = devnames['Andrej Csiaki']
devlistmb = devnames['Melissa Bledsoe']

### Create 2 dimension df, 1 column with the list, second column value fixed de Mgr ###
df6 = pd.DataFrame(devlistmc, columns=['Developer'])
df6.loc[df6['Developer']=="Pablo (Paul) Iacovone",'Developer']="Pablo Iacovone"
df6.loc[df6['Developer']=="Santiago Akira Kitashima",'Developer']="Santiago Kitashima"
df6.loc[df6['Developer']=="Ezequiel Ricardo Ferlauto",'Developer']="Ezequiel Ferlauto"
df6.loc[df6['Developer']=="Andrés Lopez Grosman",'Developer']="Andres Grosman"
df6.loc[df6['Developer']=="Dario Cesar Atach",'Developer']="Dario Atach"
df6['Manager'] = "Martin Campos"

df7 = pd.DataFrame(devlistmt, columns=['Developer'])
df7.loc[df7['Developer']=="Viera Babčancová",'Developer']="Viera Babcancova"
df7.loc[df7['Developer']=="Milan Babčanec",'Developer']="Milan Babcanec"
df7.loc[df7['Developer']=="Adam Balák",'Developer']="Adam Balak"
df7.loc[df7['Developer']=="Peter Blahut",'Developer']="Peter Blahut"
df7.loc[df7['Developer']=="Viera Csiaki",'Developer']="Viera Csiaki"
df7.loc[df7['Developer']=="Barbora Depešová",'Developer']="Barbora Depesova"
df7.loc[df7['Developer']=="Richard Hantke",'Developer']="Richard Hantke"
df7.loc[df7['Developer']=="Lenka Holáňová",'Developer']="Lenka Holanova"
df7.loc[df7['Developer']=="Andrea Hrčková",'Developer']="Andrea Hrckova"
df7.loc[df7['Developer']=="Miroslav Kocian",'Developer']="Miroslav Kocian"
df7.loc[df7['Developer']=="Kristína Margetinová",'Developer']="Kristina Margetinova"
df7.loc[df7['Developer']=="Lucia Mikuš",'Developer']="Lucia Mikus"
df7.loc[df7['Developer']=="Soňa Olšová",'Developer']="Sona Olsova"
df7.loc[df7['Developer']=="Marta Pancotti",'Developer']="Marta Pancotti"
df7.loc[df7['Developer']=="Nina Tadlánková",'Developer']="Nina Tadlankova"
df7.loc[df7['Developer']=="Ivan Trubač",'Developer']="Ivan Trubac"
df7.loc[df7['Developer']=="Kristína Vojtová",'Developer']="Kristina Vojtova"
df7.loc[df7['Developer']=="Lubomir Cmiko (Lubo)",'Developer']="Lubomir Cmiko"
df7.loc[df7['Developer']=="Radomíra Šulová",'Developer']="Radomira Sulova"
df7['Manager'] = "Marek Tarkos"

df8 = pd.DataFrame(devlistac, columns=['Developer'])
df8.loc[df8['Developer']=="Marián Chrást",'Developer']="Marian Chrast"
df8.loc[df8['Developer']=="Massimo Cireddu",'Developer']="Massimo Cireddu"
df8.loc[df8['Developer']=="Katarina Kadlecova",'Developer']="Katarina Kadlecova"
df8.loc[df8['Developer']=="Marta Kapolková",'Developer']="Marta Kapolkova"
df8.loc[df8['Developer']=="Nikolaj Kniha",'Developer']="Nikolaj Kniha"
df8.loc[df8['Developer']=="Veronika Krchnáková",'Developer']="Veronika Krchnakova"
df8.loc[df8['Developer']=="Michele Lobina",'Developer']="Michele Lobina"
df8.loc[df8['Developer']=="Vladimír Maťo",'Developer']="Vladimir Mato"
df8.loc[df8['Developer']=="Veronika Miklovičová",'Developer']="Veronika Miklovicova"
df8.loc[df8['Developer']=="Richard Nerodolík",'Developer']="Richard Nerodolik"
df8.loc[df8['Developer']=="Micheál O'Reilly",'Developer']="Mícheál O'Reilly"
df8.loc[df8['Developer']=="Michala Ožóg",'Developer']="Michala Ozog"
df8.loc[df8['Developer']=="Branislav Podoba",'Developer']="Branislav Podoba"
df8.loc[df8['Developer']=="Andre Daniel Rodrigues",'Developer']="Andre Rodrigues"
df8.loc[df8['Developer']=="Terézia Straková",'Developer']="Terezia Strakova"
df8.loc[df8['Developer']=="Lenka Tarkošová",'Developer']="Lenka Tarkosova"
df8.loc[df8['Developer']=="Valentína Vargová",'Developer']="Valentina Vargova"
df8.loc[df8['Developer']=="Marek Štípala",'Developer']="Marek Stipala"
df8['Manager'] = "Andrej Csiaki"

df9 = pd.DataFrame(devlistmb, columns=['Developer'])
df9.loc[df9['Developer']=="Gabe Hutson",'Developer']="Gabriel Hutson"
df9['Manager'] = "Melissa Bledsoe"


### Append results into one df - df10 ####
df10 = df6
df10 = df10.append(df7)
df10 = df10.append(df8)
df10 = df10.append(df9)

#pos = []
#valor = []
#cont=0

#for i in range(df5['Developer'].size):
#    x=re.search(",",df5['Developer'][i])
#    if x is not None:
#        valor.append(df5['Developer'][i])
#        Devel = valor[cont].split(",")
#        df5['Developer'][i]=Devel[0]
#        pos.append(i)
#        cont+=1

#df5 = df5.merge(df10, on="Developer", how="left")
#df5.loc[df5["ROBOT_NUMBER"]=="ODA112","Developer"]="Bruno Secchiari"
#df5.loc[df5["ROBOT_NUMBER"]=="ODA127","Developer"]="Bruno Secchiari"
#for t in range(len(pos)):
#    df5['Developer'][pos[t]]=valor[t]
    
#df5['Count']=1
        
#df5 = df5.loc[:,['Manager','Developer','Count','SAVING HS','QUARTER','ROBOT_NUMBER','TITLE','STATUS','AUTOMATION TYPE','REGIONS']]
#df5.to_excel(r"C:\Users\016434613\Desktop\Delivery Report.xlsx", index=False)

'C:\\Users\\016434613\\Downloads\\progress_dt_tasks_details_report.xlsx'